In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate();

  Activating project at `c:\Users\hario\Documents\GitHub\TinyMPC-AL\julia\bicycle_tvlqr`
  Installing known registries into `C:\Users\hario\.julia`
       Added `General` registry to C:\Users\hario\.julia\registries
┌ Warning: The active manifest file has dependencies that were resolved with a different julia version (1.10.2). Unexpected behavior may occur.
└ @ nothing C:\Users\hario\Documents\GitHub\TinyMPC-AL\julia\bicycle_tvlqr\Manifest.toml:0
   Installed x265_jll ──────────────────────────── v3.5.0+0
   Installed JpegTurbo_jll ─────────────────────── v3.0.4+0
   Installed libfdk_aac_jll ────────────────────── v2.0.3+0
   Installed libdecor_jll ──────────────────────── v0.2.2+0
   Installed OrdinaryDiffEqRKN ─────────────────── v1.1.0
   Installed OffsetArrays ──────────────────────── v1.14.2
   Installed OrdinaryDiffEqStabilizedRK ────────── v1.1.0
   Installed MutableArithmetics ────────────────── v1.6.0
   Installed Libmount_jll ──────────────────────── v2.40.2+0
   Installed Hy

In [15]:
using Symbolics
using LinearAlgebra
import ForwardDiff as FD

In [18]:
# ====================
# Explicit model params (working)
# State: x, y, theta
# Input: v, delta
# ====================
@variables x[1:7] u[1:7]        ##################### CHANGE NUMBER OF STATES AND INPUTS HERE
x = collect(x); u = collect(u);
yaw0 = 0.72; # choose some initial yaw angle
x0 = [-0.003; +0.001; yaw0; cos(yaw0); sin(yaw0); 0.0; 0.0];
# ... some issues on the solver end, need padding states to make the state dimension equal to the input dimension
# x0 = rand(3); # use for 3-state model
# u0 = [0.1; 0.3]; # use for 2-input model 
u0 = rand(7); # use for 16-input model

#= # CLARI params
kinematic_v_dot = 2.85e-3; # m/s
kinematic_theta_dot = 0.950; # rad/s =#

# gait params
phiFreq = 5; # Hz
H_phi = 1/phiFreq; # s (set to gait frequency) # previous values: 0.1, 0.05
###### NEED TO ENSURE THAT H IS AN EVEN MULTIPLE OF H_phi ######
# this makes the SE2 dynamics integrator work correctly with midpointSE2 and rk4SE2 functions
solverFreq = 2.5; # Hz
H = 1/solverFreq; # s (set to TinyMPC frequency-- two gait cycles are complete for one control update)
# previous values: 0.2, 0.1, 0.05
N = H/H_phi; # number of gait cycles to integrate over when stepping through the dynamics
# ==================== 2024/04/28: 11:10AM gait frequency = 10Hz and TinyMPC frequency = 5Hz; H = 0.2; 
# ====================                                                ODE Integration Scheme: Midpoint
#= # CLARI kinematic model (similar to Dubins)
# ... robot dynamics in the rest frame
function dynamics(x, u) 

    θ = x[3]  # yaw

    c = cos(θ)
    s = sin(θ)

    ẋ = kinematic_v_dot*u[1]*c
    ẏ = kinematic_v_dot*u[1]*s
    θ̇ = kinematic_theta_dot*u[2]

    return [ẋ, ẏ, θ̇]
end =#

#= # CLARI kinematic model with S1 yaw rate (05/01/2024)
# ... robot dynamics in the rest frame
# ... first two states are planar locations x and y
# ... third and fourth states are cos(yaw) and sin(yaw) respectively
# ... this makes the problem respect the group strucutre of SE(2)
# ... you won't have +-pi discontinuities in the yaw angle 
# ... ... and impulses through the solver
function dynamics(x, u)

    # initialize positions
    x_pos   = x[1]    # x position
    y_pos   = x[2]    # y position
    cos_theta  = x[3]    # cos(yaw) (heading unit vector projection on x-axis)
    sin_theta  = x[4]    # sin(yaw) (heading unit vector projection on y-axis)

    # initialize velocities in rest frame
    x_dot           = kinematic_v_dot*u[1]*cos_theta        # x velocity
    y_dot           = kinematic_v_dot*u[1]*sin_theta        # y velocity
    cos_theta_dot   = -kinematic_theta_dot*u[2]*sin_theta   # cos(yaw) velocity
    sin_theta_dot   =  kinematic_theta_dot*u[2]*cos_theta   # sin(yaw) velocity

    # return the dynamics
    return [x_dot, y_dot, cos_theta_dot, sin_theta_dot]

end =#

#= # CLARI kinematic model with S1 yaw rate (05/03/2024)
# ... same as the previous model, but with an additional input to control 
# ... all of the swing amplitudes simultaneously
function dynamics(x, u)

    # initialize positions
    x_pos   = x[1]    # x position
    y_pos   = x[2]    # y position
    cos_theta  = x[3]    # cos(yaw) (heading unit vector projection on x-axis)
    sin_theta  = x[4]    # sin(yaw) (heading unit vector projection on y-axis)

    # initialize velocities in rest frame
    x_dot           =  u[3]*kinematic_v_dot*u[1]*cos_theta        # x velocity
    y_dot           =  u[3]*kinematic_v_dot*u[1]*sin_theta        # y velocity
    cos_theta_dot   = -u[3]*kinematic_theta_dot*u[2]*sin_theta    # cos(yaw) velocity
    sin_theta_dot   =  u[3]*kinematic_theta_dot*u[2]*cos_theta    # sin(yaw) velocity

    # return the dynamics
    return [x_dot, y_dot, cos_theta_dot, sin_theta_dot]

end =#

#################################################################### SINDy models %%
#= # CLARI kinematic model with S1 yaw rate (06/01/2024)
# ... SINDy first-order, TVRefDiff, cycle-averaged dynamics model
function dynamics(x, u)

    # initialize positions
    x_pos   = x[1]    # x position
    y_pos   = x[2]    # y position
    cos_theta  = x[3]    # cos(yaw) (heading unit vector projection on x-axis)
    sin_theta  = x[4]    # sin(yaw) (heading unit vector projection on y-axis)

    # initialize velocities in body frame
    xi_x           = -0.0773*u[1] + 0.8668*u[2] + 0.0263 # body x velocity
    xi_y           = -0.0130*u[1] - 0.9974*u[2] - 0.0008 # body y velocity
    xi_theta       =  0.0437*u[1] - 0.5123*u[2] - 0.0010 # body yaw rate

    # initialize velocities in rest frame
    x_dot           =  xi_x*cos_theta - xi_y*sin_theta         # x velocity
    y_dot           =  xi_x*sin_theta + xi_y*cos_theta         # y velocity
    cos_theta_dot   = -x[4]*xi_theta                           # cos(yaw) velocity
    sin_theta_dot   =  x[3]*xi_theta                           # sin(yaw) velocity

    # return the dynamics
    return [x_dot, y_dot, cos_theta_dot, sin_theta_dot]

end =#

#= # CLARI kinematic model (07/21/2024)
# ... SINDy first-order, finite-difference, cycle-averaged dynamics model
function dynamics(x, u)

    # initialize positions
    theta  = x[3]              # robot heading angle
    cos_theta  = cos(theta)    # cos(theta) (heading unit vector projection on x-axis)
    sin_theta  = sin(theta)    # sin(theta) (heading unit vector projection on y-axis)

    # initialize velocities in body frame
    xi_x           = 0.003618*u[2]- 0.003872*u[1] + 0.007932*u[3]+ 0.002661*u[4]+ 0.004506*u[5] - 0.003487*u[6]+ 0.006692*u[7]+ 0.002624*u[8]- 0.01506*u[9]+ 0.01044*u[10] - 0.01735*u[11] - 0.007047*u[12]+ 4.002e-5*u[13]+ 0.005165*u[14]- 0.001332*u[15]- 0.00485*u[16]+ 0.00538 # body x velocity
    xi_y           = 0.001384*u[1]- 0.0001855*u[2]+ 2.221e-6*u[3]+ 0.0004202*u[4]+ 0.0007788*u[5] + 0.0002786*u[6]+ 3.928e-5*u[7]+ 0.0004973*u[8]- 0.001249*u[9]- 8.014e-5*u[10]- 0.001685*u[11] - 0.0003461*u[12]- 0.000237*u[13]+ 0.0001387*u[14]- 0.0002612*u[15]+ 0.0007736*u[16]- 0.000519 # body y velocity
    xi_theta       = 1.019*u[1]- 1.221*u[2]- 0.2307*u[3]+ 2.032*u[4]- 0.09099*u[5] + 0.5205*u[6]+ 0.9639*u[7]+ 0.5104*u[8]- 0.4504*u[9]+ 0.9095*u[10]- 1.513*u[11] - 2.305*u[12]- 0.2794*u[13]+ 1.852*u[14]- 0.6948*u[15]+ 0.5795*u[16]- 2.069 # body yaw rate

    # initialize velocities in rest frame
    x_dot           =  xi_x*cos_theta - xi_y*sin_theta         # x velocity in rest frame
    y_dot           =  xi_x*sin_theta + xi_y*cos_theta         # y velocity in rest frame
    theta_dot       =  xi_theta                                # yaw rate in rest frame

    # return the dynamics
    return [x_dot, y_dot, theta_dot]

end =#

# 2-input CLARI model generated on (07/06/2024) -- FINITE DIFFERENCE
# xi_x           = -0.0773*u[1] + 0.8668*u[2] + 0.0263 
# xi_y           = -0.0130*u[1] - 0.9974*u[2] - 0.0008
# xi_theta       =  0.0437*u[1] - 0.5123*u[2] - 0.0010 

# 16-input mCLARI model generated on (07/21/2024) -- FINITE DIFFERENCE
# 0.003618*u[2]- 0.003872*u[1]+ 0.007932*u[3]+ 0.002661*u[4]+ 0.004506*u[5]- 0.003487*u[6]+ 0.006692*u[7]+ 0.002624*u[8]- 0.01506*u[9]+ 0.01044*u[10]- 0.01735*u[11]- 0.007047*u[12]+ 4.002e-5*u[13]+ 0.005165*u[14]- 0.001332*u[15]- 0.00485*u[16]+ 0.00538
# 0.001384*u[1]- 0.0001855*u[2]+ 2.221e-6*u[3]+ 0.0004202*u[4]+ 0.0007788*u[5]+ 0.0002786*u[6]+ 3.928e-5*u[7]+ 0.0004973*u[8]- 0.001249*u[9]- 8.014e-5*u[10]- 0.001685*u[11]- 0.0003461*u[12]- 0.000237*u[13]+ 0.0001387*u[14]- 0.0002612*u[15]+ 0.0007736*u[16]- 0.000519
# 1.019*u[1]- 1.221*u[2]- 0.2307*u[3]+ 2.032*u[4]- 0.09099*u[5]+ 0.5205*u[6]+ 0.9639*u[7]+ 0.5104*u[8]- 0.4504*u[9]+ 0.9095*u[10]- 1.513*u[11]- 2.305*u[12]- 0.2794*u[13]+ 1.852*u[14]- 0.6948*u[15]+ 0.5795*u[16]- 2.069

# 16-input mCLARI model generated on (07/21/2024) -- FINITE DIFFERENCE
# 0.003536*u[2]- 0.00323*u[1]+ 0.008277*u[3]+ 0.002455*u[4]+ 0.005199*u[5]- 0.00336*u[6]+ 0.006394*u[7]+ 0.002023*u[8]- 0.01502*u[9]+ 0.01079*u[10]- 0.01782*u[11]- 0.006954*u[12]- 0.0001218*u[13]+ 0.005277*u[14]- 0.001471*u[15]- 0.004749*u[16]+ 0.00465
# 0.001043*u[1]- 0.000151*u[2]- 7.221e-5*u[3]+ 0.0003382*u[4]+ 0.0006829*u[5]+ 0.0002047*u[6]+ 4.856e-5*u[7]+ 0.0005452*u[8]- 0.0008777*u[9]+ 0.0001556*u[10]- 0.001452*u[11]- 0.0001217*u[12]- 0.0002166*u[13]+ 5.541e-6*u[14]- 0.0003422*u[15]+ 0.000777*u[16]- 0.0005358
# 1.852*u[2]- 2.786*u[1]+ 2.33*u[3]- 1.586*u[4]+ 1.316*u[5]+ 0.12*u[6]- 2.077*u[7]+ 0.509*u[8]+ 4.203*u[9]+ 0.535*u[10]- 1.419*u[11]+ 4.094*u[12]+ 0.08559*u[13]- 3.836*u[14]+ 0.2467*u[15]- 2.8*u[16]- 0.06712

#= # HAMR kinematic model with S1 yaw rate (12/17/2024)
# ... SINDy first-order, increasing-order central difference derivative, cycle-averaged dynamics model
function dynamics(x, u)

    # initialize positions
    x_pos   = x[1]    # x position
    y_pos   = x[2]    # y position
    cos_theta  = x[3]    # cos(yaw) (heading unit vector projection on x-axis)
    sin_theta  = x[4]    # sin(yaw) (heading unit vector projection on y-axis)

    # initialize the input vector
    U = [u[1]; u[2]; u[3]];

    # initialize velocities in body frame as a function of the inputs
        # model 1 - B matrix
    Xi             = [0.01537 0.002067 0.0006823;
                      0.03835 0.01061  0.005287;
                      0.1564  0.09182  0.06229]*U;
#=         # model 2
    Xi             = [-0.03167, 0.001426, -0.003682; 
                       0.02373, 0.007228, -0.003643; 
                       0.001715, -0.003266, -0.0007891]*U;
        # model 3
    Xi             = [-0.009649 -0.001884 -0.001667; 
                       0.04422 0.006511 0.001405; 
                       0.1172 0.07258 0.04917]*U; =#

    # unpack the body velocities
    xi_x           = Xi[1] # body x velocity
    xi_y           = Xi[2] # body y velocity
    xi_theta       = Xi[3] # body yaw rate

    # initialize velocities in rest frame
    x_dot           =  xi_x*cos_theta - xi_y*sin_theta         # x velocity
    y_dot           =  xi_x*sin_theta + xi_y*cos_theta         # y velocity
    cos_theta_dot   = -x[4]*xi_theta                           # cos(yaw) velocity
    sin_theta_dot   =  x[3]*xi_theta                           # sin(yaw) velocity

    # return the dynamics
    return [x_dot, y_dot, cos_theta_dot, sin_theta_dot]

end =#

# HAMR kinematic model with S1 yaw rate (12/23/2024)
# ... SINDy first-order, increasing-order central difference derivative, cycle-averaged dynamics model
# ... padded states and inputs to make the state dimension equal to the input dimension
function dynamics(x, u)

    # initialize positions - 3-STATE VERSION
    x_pos       = x[1]              # x position
    y_pos       = x[2]              # y position
    theta_pos   = x[3]              # yaw
    cos_theta   = x[4]              # cos(yaw) (heading unit vector projection on x-axis)
    sin_theta   = x[5]              # sin(yaw) (heading unit vector projection on y-axis)

    # initialize the input vector
    U = [ u[1]; u[2]; u[3]; u[4]; u[5]; u[6]; u[7] ];

    # initialize velocities in body frame as a function of the inputs
    Xi             = [ -0.0014	-0.0026	0.0001	-0.0021	-0.0011	0.0027	0.0089;
                        0.0054	-0.0262	0.0039	-0.0150	-0.0062	-0.0050	0.0021;
                        0.9064	-1.0593	0.9922	-1.8390	0.6479	1.3418	-0.1387]*U;

    # unpack the body velocities
    xi_x           =  Xi[1]             # body x velocity
    xi_y           =  Xi[2]             # body y velocity
    xi_theta       =  Xi[3]             # yaw rate
    xi_cos_theta   = -x[5]*Xi[3]        # cos(yaw) velocity (orientation phasor velocity in the x-direction)
    xi_sin_theta   =  x[4]*Xi[3]        # sin(yaw) velocity (orientation phasor velocity in the y-direction)

    # initialize velocities in rest frame
    # ... only the translational velocities are pushedforward to the rest frame
    x_dot           =  xi_x*cos_theta - xi_y*sin_theta         # world x velocity
    y_dot           =  xi_x*sin_theta + xi_y*cos_theta         # world y velocity
    theta_dot       =  xi_theta                                # yaw rate
    cos_theta_dot   =  xi_cos_theta                            # cos(yaw) velocity
    sin_theta_dot   =  xi_sin_theta                            # sin(yaw) velocity

    # return the dynamics
    # ... last two pad states have no dynamics
    return [x_dot, y_dot, theta_dot, cos_theta_dot, sin_theta_dot, 0.0, 0.0]

end

# ==================== ode integrators for continuous dynamics ====================
function fwdEuler(x, u)
    x += H*dynamics(x, u)
end
function midpoint(x, u)
    dt = H
    k1 = dt*dynamics(x, u)
    k2 = dt*dynamics(x + k1/2, u)
    x += k2
end
function rk4(x, u)
    dt = 0.1
    k1 = dt*dynamics(x, u)
    k2 = dt*dynamics(x + k1/2, u)
    k3 = dt*dynamics(x + k2/2, u)
    k4 = dt*dynamics(x + k3, u)
    x += (1/6)*(k1 + 2*k2 + 2*k3 + k4)
end
# =================================================================================

# ==================== ode integrators for cycle-averaged dynamics ================
# the gait modified versions of the ode integrators are made to respect the cycle averaged dynamics
# ... midpoint and RK4 are still INCORRECT in a gait-avergaed sense, but they still could have practical utility
# ... fwdEuler is CORRECT in a gait-averaged sense
function fwdEulerGaitAvg(x, u)
    x += integrateGaitAvgdynamics(x, u, H, N, dynamics)
end
function midpointGaitAvg(x, u)
    k1 = integrateGaitAvgdynamics(x, u, H, N, dynamics)
    k2 = integrateGaitAvgdynamics(x + k1/2, u, H, N, dynamics)
    x += k2
end
function rk4GaitAvg(x, u)
    # H: discrete dynamics time step (nominally as long as the tinyMPC update period)
    k1 = integrateGaitAvgdynamics(x,        u, H, N, dynamics)
    k2 = integrateGaitAvgdynamics(x + k1/2, u, H, N, dynamics)
    k3 = integrateGaitAvgdynamics(x + k2/2, u, H, N, dynamics)
    k4 = integrateGaitAvgdynamics(x + k3,   u, H, N, dynamics)
    x += (1/6)*(k1 + 2*k2 + 2*k3 + k4)
end
# =================================================================================

# ==================== integrate position over multiple cycles ====================
# helps integrate the dynamics over N gait cycles-- can't integrate this normally because the time-steps
# ... are quantized to the gait cycle period: T, 2T, 3T, ..., NT
function integrateGaitAvgdynamics(x, u, dt, N, dynSE2)
    # N: number of gait cycles to integrate over 
    # dt is the total time for N gait cycles
    # x is the current state and u is the current Input
    # dynSE2 is the function that provides the dynamics wrt to the origin
    if N > 1
        dtNow = dt/N # time period for each gait cycle
        delx = zeros(length(x0)) # initialize the state increment # 3-state version now
        for i = 1:N
            delx += dtNow*dynSE2(x + delx, u) # integrate the dynamics
        end
        return delx # return the state increment
    else
        return dt*dynSE2(x, u) # return the state increment (only one time period)
    end
end
# =================================================================================

# ====================
# Jacobian calculation
# ... doing it normally without simplification
# ... doing it with just a simplify call
# ... doing it with a simplify call and no expansion and trying to reduce it all the way
#= sym_jac_A = Symbolics.jacobian(fwdEulerGaitAvg(x, u), x)
sym_jac_B = Symbolics.jacobian(fwdEulerGaitAvg(x, u), u) =#
#= sym_jac_A = simplify( Symbolics.jacobian(fwdEulerGaitAvg(x, u), x) )
sym_jac_B = simplify( Symbolics.jacobian(fwdEulerGaitAvg(x, u), u) ) =#
sym_jac_A = simplify( Symbolics.jacobian(fwdEulerGaitAvg(x, u), x); expand = false,
                                                                    rewriter = nothing )
sym_jac_B = simplify( Symbolics.jacobian(fwdEulerGaitAvg(x, u), u); expand = false,
                                                                    rewriter = nothing )
#= sym_jac_A = Symbolics.jacobian(midpointGaitAvg(x, u), x)
sym_jac_B = Symbolics.jacobian(midpointGaitAvg(x, u), u) =#
#= sym_jac_A = Symbolics.jacobian(rk4GaitAvg(x, u), x)
sym_jac_B = Symbolics.jacobian(rk4GaitAvg(x, u), u) =#
#= sym_jac_A = Symbolics.jacobian(rk4(x, u), x)
sym_jac_B = Symbolics.jacobian(rk4(x, u), u) =#

# Sub value to validate with ForwardDiff
sub_dict = Dict()
[sub_dict[x] = y for (x,y) in zip([x; u], [x0; u0])]
val_jac_A = substitute.(sym_jac_A, (sub_dict,))
val_jac_B = substitute.(sym_jac_B, (sub_dict,))

# calculate the jacobian using ForwardDiff
fd_jac_A = FD.jacobian(_x->fwdEulerGaitAvg(_x, u0), x0)
fd_jac_B = FD.jacobian(_u->fwdEulerGaitAvg(x0, _u), u0)
#= fd_jac_A = FD.jacobian(_x->midpointGaitAvg(_x, u0), x0)
fd_jac_B = FD.jacobian(_u->midpointGaitAvg(x0, _u), u0) =#
#= fd_jac_A = FD.jacobian(_x->rk4GaitAvg(_x, u0), x0)
fd_jac_B = FD.jacobian(_u->rk4GaitAvg(x0, _u), u0) =#
#= fd_jac_A = FD.jacobian(_x->rk4(_x, u0), x0)
fd_jac_B = FD.jacobian(_u->rk4(x0, _u), u0) =#

# display the results (compare the jacobians)
display(norm(fd_jac_A - val_jac_A) + norm(fd_jac_B - val_jac_B))
display(val_jac_A)
display(val_jac_B)

# display the symbolic jacobians
display(sym_jac_A)
display(sym_jac_B)

3.0681178646504203e-16

7×7 Matrix{Num}:
 1  0  0    0.0037882  0.00434761  0  0
 0  1  0  -0.00434761   0.0037882  0  0
 0  0  1            0           0  0  0
 0  0  0     0.937899   -0.498402  0  0
 0  0  0     0.498402    0.937899  0  0
 0  0  0            0           0  1  0
 0  0  0            0           0  0  1

7×7 Matrix{Num}:
 -0.00186927   0.00704097  …  0.00166754   0.00242525  0.00173112
  0.00152622  -0.00838716     -0.0016336  0.000216406  0.00316666
     0.36256     -0.42372        0.25916      0.53672    -0.05548
   -0.306992     0.358779       -0.21944     -0.45446   0.0469769
    0.212999     -0.24893       0.152253     0.315316  -0.0325937
           0            0  …           0            0           0
           0            0              0            0           0

7×7 Matrix{Num}:
 1  0  0  …  0  0
 0  1  0     0  0
 0  0  1     0  0
 0  0  0     0  0
 0  0  0     0  0
 0  0  0  …  1  0
 0  0  0     0  1

7×7 Matrix{Num}:
 0.2(-0.0014x[4] - 0.0054x[5]) + 0.2(-0.18128(0.0054u[1] - 0.0262u[2] + 0.0039u[3] - 0.015u[4] - 0.0062u[5] - 0.005u[6] + 0.0021u[7])*x[4] + 0.0054(-x[5] - 0.2(0.9064u[1] - 1.0593u[2] + 0.9922u[3] - 1.839u[4] + 0.6479u[5] + 1.3418u[6] - 0.1387u[7])*x[4]) - 0.18128(-0.0014u[1] - 0.0026u[2] + 0.0001u[3] - 0.0021u[4] - 0.0011u[5] + 0.0027u[6] + 0.0089u[7])*x[5] - 0.0014(x[4] - 0.2(0.9064u[1] - 1.0593u[2] + 0.9922u[3] - 1.839u[4] + 0.6479u[5] + 1.3418u[6] - 0.1387u[7])*x[5]))  …  0.2(0.0089x[4] - 0.0021x[5]) + 0.2(0.02774(0.0054u[1] - 0.0262u[2] + 0.0039u[3] - 0.015u[4] - 0.0062u[5] - 0.005u[6] + 0.0021u[7])*x[4] + 0.0021(-x[5] - 0.2(0.9064u[1] - 1.0593u[2] + 0.9922u[3] - 1.839u[4] + 0.6479u[5] + 1.3418u[6] - 0.1387u[7])*x[4]) + 0.02774(-0.0014u[1] - 0.0026u[2] + 0.0001u[3] - 0.0021u[4] - 0.0011u[5] + 0.0027u[6] + 0.0089u[7])*x[5] + 0.0089(x[4] - 0.2(0.9064u[1] - 1.0593u[2] + 0.9922u[3] - 1.839u[4] + 0.6479u[5] + 1.3418u[6] - 0.1387u[7])*x[5]))
   0.2(0.0054x[4] - 0.0014x[

In [19]:
# ====================
# Generate C code for jacobians
# ====================
#= fA = build_function(sym_jac_A, x, u; target=Symbolics.CTarget(), 
    fname = :tiny_Bicycle3dGetJacobianA_Raw, lhsname=:A, rhsnames=[Symbol("x"), Symbol("u")]);

open("functionA.txt","a") do io
        println(io, fA)
end

fB = build_function(sym_jac_B, x, u; target=Symbolics.CTarget(), 
    fname = :tiny_Bicycle3dGetJacobianB_Raw, lhsname=:B, rhsnames=[Symbol("x"), Symbol("u")]);

open("functionB.txt","a") do io
        println(io, fB)
end =#


fA = build_function(sym_jac_A, x, u; target=Symbolics.CTarget(), 
    fname = :tiny_HAMRv6GetJacobianA_Raw, lhsname=:A, rhsnames=[Symbol("x"), Symbol("u")]);

open("functionA.txt","a") do io
        println(io, fA)
end

fB = build_function(sym_jac_B, x, u; target=Symbolics.CTarget(), 
    fname = :tiny_HAMRv6GetJacobianB_Raw, lhsname=:B, rhsnames=[Symbol("x"), Symbol("u")]);

open("functionB.txt","a") do io
        println(io, fB)
end

In [20]:
# ====================
# Generate C code for dynamics
# ====================

# ==================== continuous tine dynamics ====================

#= dynamics_fwdEuler = fwdEuler(x ,u)
dynamics_fwdEuler = build_function(dynamics_fwdEuler, x, u; target=Symbolics.CTarget(), 
    fname = :tiny_CLARI3dNonlinearDynamics_Raw, lhsname=:xn, rhsnames=[Symbol("x"), Symbol("u")])
open("dynamics.txt","a") do io
        println(io, dynamics_fwdEuler)
end =#

#= dynamics_midpoint = midpoint(x ,u)
dynamics_midpoint = build_function(dynamics_midpoint, x, u; target=Symbolics.CTarget(), 
    fname = :tiny_CLARI3dNonlinearDynamics_Raw, lhsname=:xn, rhsnames=[Symbol("x"), Symbol("u")])
open("dynamics.txt","a") do io
        println(io, dynamics_midpoint)
end =#

#= dynamics_rk4 = rk4(x ,u)
dynamics_rk4 = build_function(dynamics_rk4, x, u; target=Symbolics.CTarget(), 
    fname = :tiny_CLARI3dNonlinearDynamics_Raw, lhsname=:xn, rhsnames=[Symbol("x"), Symbol("u")])
open("dynamics.txt","a") do io
        println(io, dynamics_rk4)
end =#

# ==================== adjusted to work with gait cycle averaged dynamics

dynamics_fEulerGA = fwdEulerGaitAvg(x ,u)
dynamics_fEulerGA = build_function(dynamics_fEulerGA, x, u; target=Symbolics.CTarget(), 
                fname = :tiny_HAMRv6NonlinearDynamics_Raw, lhsname=:xn, rhsnames=[Symbol("x"), Symbol("u")])
open("dynamics.txt","a") do io
        println(io, dynamics_fEulerGA)
end

#= dynamics_midptGA = midpointGaitAvg(x ,u)
dynamics_midptGA = build_function(dynamics_midptGA, x, u; target=Symbolics.CTarget(), 
                fname = :tiny_CLARI3dNonlinearDynamics_Raw, lhsname=:xn, rhsnames=[Symbol("x"), Symbol("u")])
open("dynamics.txt","a") do io
        println(io, dynamics_midptGA)
end =#

#= dynamics_rk4GA = rk4GaitAvg(x ,u)
dynamics_rk4GA = build_function(dynamics_rk4GA, x, u; target=Symbolics.CTarget(), 
                fname = :tiny_CLARI3dNonlinearDynamics_Raw, lhsname=:xn, rhsnames=[Symbol("x"), Symbol("u")])
open("dynamics.txt","a") do io
        println(io, dynamics_rk4GA)
end =#

In [ ]:
# ====================
# Symbolic model params
# not working 
# ====================
@variables g, m, ℓ, J
model = (g=g, m=m, ℓ=ℓ, J=J)
x = collect(x)
u = collect(u)
@variables x[1:6] u[1:2] 

# Planar Quadrotor Dynamics
function dynamics(model, x, u)
    θ = x[3]
    ẍ = (1/model.m)*(u[1] + u[2])*sin(θ)
    ÿ = (1/model.m)*(u[1] + u[2])*cos(θ) - model.g
    θ̈ = (1/model.J)*(model.ℓ/2)*(u[2] - u[1])
    
    return [x[4:6]; ẍ; ÿ; θ̈]
end
function rk4(model, x, u)
    dt = 0.1
    k1 = dt*dynamics(model, x, u)
    k2 = dt*dynamics(model, x + k1/2, u)
    k3 = dt*dynamics(model, x + k2/2, u)
    k4 = dt*dynamics(model, x + k3, u)
    x + (1/6)*(k1 + 2*k2 + 2*k3 + k4) 
end
sym_jac_A = Symbolics.jacobian(dynamics(model, x, u), x)
sym_jac_B = Symbolics.jacobian(dynamics(model, x, u), u)
# sym_jac_A = Symbolics.jacobian(rk4(model, x, u), u)

In [ ]:
function lqr_controller(t, x)
    #LQR Hover Controller
    A = FD.jacobian(_x->rk4(_x, u0), xref)
    B = FD.jacobian(_u->rk4(x0, _u), uref)
    K = dlqr(A,B,Q,R)
    return u_hover - K*(x-xref)
end

In [ ]:
function closed_loop(x0,controller,N)
    xhist = zeros(length(x0),N)
    u0 = controller(1,x0)
    uhist = zeros(length(u0),N-1)
    uhist[:,1] .= u0
    xhist[:,1] .= x0
    for k = 1:(N-1)
        uk = controller(k,xhist[:,k], xref[k], uref[k])
        uhist[:,k] = max.(min.(umax, uk), umin) #enforce control limits
        xhist[:,k+1] .= rk4(xhist[:,k],uhist[:,k])
    end
    return xhist, uhist
end

In [ ]:
nx = 3; nu = 2; N = 200;
uhist = [[1; 0.2] for i = 1:N-1]
xhist = [zeros(nx) for i = 1:N]
xhist[1] .= [0; 0; 0.0]
for k = 1:N-1
  xhist[k+1] = rk4(xhist[k], uhist[k])
end
display(xhist)
# export_vec_to_c("data", "sfloat", "U_ref_data", uhist)

In [ ]:
x = [[1,2,3,4],[5,6,7,8]]

# Export a vector of vectors to C header and source 
function export_vec_to_c(filename, var_type, var_name, x)
    declare = var_type * " " * var_name * "[" * string(length(x)*length(x[1])) * "]"
    def = declare * " = {\n"
    for i=1:length(x)
        def = def * "  "
        for j=1:length(x[1])
            def = def * string(x[i][j])
            if j < length(x[1]) 
                def = def * ","
            end
        end
        if i < length(x)
            def = def * ",\n"
        end
    end
    def = def*"}"

    # open(filename*".h","a") do io
    #         println(io, declare * ";\n")
    # end
    open(filename*".h","a") do io
            println(io, def * ";\n")
    end
    return true
end
export_vec_to_c("data", "sfloat", "X_ref_data", xhist)

In [ ]:
function get_jacobians!(A, B, f, x0, u0)
    @assert size(A) == (Nx, Nx)
    @assert size(B) == (Nx, Nu)
    @assert size(f) == (Nx,)
    A .= FD.jacobian(_x->rk4(_x, u0), x0)
    B .= FD.jacobian(_u->rk4(x0, _u), u0)
    f .= rk4(x0, u0) - rk4(x0, u0)
end
function tiny_Riccati_LTV!(params, A::Matrix, B::Matrix, f::Vector, 
        get_jacobians::Function, X, U, K::Vector, d::Vector, P::Vector, p::Vector,μ,ρ)::Integer
    
    Q = params.Q; R = params.R; q = params.q; r = params.r; 
    xref = params.xref; uref = params.uref;
    Nx = params.Nx; Nu = params.Nu; N = params.N
    # Copy terminal cost-to-go
    k = N
    P[k] .= Q
    p[k] .= q

    # Sxx = zeros(n, n)
    # Sxu = zeros(n, m)
    # Sux = zeros(m, n)
    # Suu = zeros(m, m)
    # Sx = zeros(n, 1)
    # Su = zeros(m, 1)

    for k = N-1:-1:1 
        # get_jacobians!(A, B, f, X[k], U[k])

        Sx = q + A' * (P[k+1] * f + p[k+1])
        Su = r + B' * (P[k+1] * f + p[k+1])
        Sxx = Q + A'*P[k+1]*A
        Suu = R + B'*P[k+1]*B
        Sxu = P[k+1] * B                
        Suu = R + B'*P[k+1]*B                         
        Sux = B'*P[k+1]*A   
        
        # control constraints
        huv = ineq_con_u(U[k])  # calculate h(u) constraint
        mask = eval_mask(μ[k], huv)  # choose active
        ∇hu = ineq_con_u_jac(U[k])
        Su  += ∇hu'*(μ[k] + ρ*(mask * huv)) # add to cost
        Suu += ρ*∇hu'*mask*∇hu
        
        d[k] .= Suu\Su
        K[k] .= Suu\Sux

        P[k] .= Sxx + K[k]'*Suu*K[k] + K[k]'*Sux + Sux'*K[k]
        p[k] .= Sx + K[k]'*Suu*d[k] + K[k]'*Su + Sux'*d[k]
    end
    
    return 0;
end
function tiny_RiccatiForwardPass_LTV!(params, A, B, f, get_jacobians::Function,
                                    K::Vector, d::Vector, P::Vector, p::Vector, X::Vector, U::Vector) 
    N = params.N
    for k=1:N-1
        # get_jacobians!(A, B, f, X[k], U[k])
        U[k] .= -K[k]*X[k] - d[k]
        X[k+1] .= A*X[k] + B*U[k]
    end
  return 0
end
function tiny_AugmentedLagrange_LQR(params, A, B, f, get_jacobians, 
        K::Vector, d::Vector, P::Vector, p::Vector, X::Vector, U::Vector)
    # first check the sizes of everything
    @assert length(X) == params.N
    @assert length(U) == params.N-1
    @assert length(X[1]) == params.nx
    @assert length(U[1]) == params.nu
    @assert length(ineq_con_u(params,U[1])) == params.ncu  # no constraint control
    @assert length(ineq_con_x(params,X[1])) == params.ncx  # no constraint state
end    
function eval_mask(μv, huv)  
    # Extract active inequality constraints
    # active set mask
    mask = Diagonal(zeros(length(huv)))
    for i = 1:length(huv)
        mask[i,i] = (μv[i] > 0 || huv[i] > 0)
    end
    mask
end
function ineq_con_u(p,u)
    [u-p.u_max;-u + p.u_min]
end
function ineq_con_u_jac(params, u)
    FD.jacobian(_u -> ineq_con_u(params,_u), u)
end

In [ ]:
using DelimitedFiles
uref1 = readdlm("uref_data.txt")
uref1 = reshape(uref1, Nu, Nt-1)
xref1 = readdlm("xref_data.txt")
xref1 = reshape(xref1, Nx, Nt)
uref = [uref1[1:Nu, i] for i = 1:(Nt-1)]
xref = [xref1[1:Nx, i] for i = 1:Nt];
params = (
    Nx = Nx,
    Nu = Nu,
    Nt = Nt,
    N = Nh,
    xref = xref,
    uref = uref,
    Q = Q,
    R = R,
    q = q,
    r = r,
    u_min = umin,
    u_max = umax,
);

In [ ]:
dt = 0.1  
h = dt
Nx = 5     # number of state
Nu = 2     # number of controls
Tfinal = 10.0 # final time
Nt = Int(Tfinal/h)+1    # number of time steps
thist = Array(range(0,h*(Nt-1), step=h));
    
# Thrust limits
umax = [2; 0.5]
umin = -umax

# Cost weights
Q = Array(1.0*I(Nx)); 
q = zeros(Nx)
R = Array(.01*I(Nu)); 
r = zeros(Nu)
Qn = Array(1.0*I(Nx));

A = zeros(Nx, Nx)
B = zeros(Nx, Nu)
f = zeros(Nx)

x0 = [3; -1; 0; 1; 1]
xf = [20.0; 20.0; pi/4; 0; 0]


In [ ]:
xhist = [zeros(Nx) for i=1:Nt]
Phist = [zeros(Nx, Nx) for i=1:Nt]
phist = [zeros(Nx) for i=1:Nt]
uhist = [zeros(Nu) for i=1:Nt-1]
Khist = [zeros(Nu, Nx) for i=1:Nt-1]
dhist = [zeros(Nu) for i=1:Nt-1];
xhist = tiny_AL_LQR(params,X,U,P,p,K,d,Xn,Un;atol=1e-1,max_iters = 3000,verbose = true,ρ = 1e0, ϕ = 10.0);